#  Potato Plant Diseases Data 🍂🍂🍂🌱🌱🌱🥔🥔🥔

Potato Plant Diseases Detection Using CNN

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt


Data Preprocessing

In [4]:
# Defining variables
BATCH_SIZE = 32
IMAGE_SIZE = 256
EPOCHS = 50
CHANNELS = 3

In [5]:
# Retrieve images and divide into three classes
dataset = tf.keras.preprocessing.image_dataset_from_directory(r'C:\Users\PC\Documents\Data-Science-Tasks\ML-Image-Classification\Potato Image Classification\PlantVillage',
                                                              shuffle = True,
                                                              image_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size = BATCH_SIZE
)


Found 2152 files belonging to 3 classes.


In [6]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']